https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [149]:
import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

In [150]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)

import re

## [00] define a state dict to contain - task, plan, steps, and other variables.

In [151]:
from typing import TypedDict, List


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

## [0] Embedding/vectorisation

In [152]:
# For RAG

embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    chunk_size=1 
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=search_endpoint,
    azure_search_key=search_key,
    index_name="boardai03",
    embedding_function=embeddings.embed_query,
)

retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 2})

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

[llm]

In [153]:
llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",    
)

## [1] Planner
https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb?ref=blog.langchain.dev

In [154]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:

(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general
world knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. 
Input will be a list of board members and agendas.

(3) COMPILE[input]:Worker that add all the separate paper into one board paper. input will be multiple board paper pieces from board members.

(4) CIRCULATE[input]: Worker that send compiled board paper to board members and get feed back and fix. Input will be a board paper. 

(5) PEOPLE_DB[input]: Worker that gives you a list of relevant people's name, email based on their role, team. Input will be role or team name.

For example,
Task: execute this: To create a board paper, first, based on an approved agenda, the secretary sends an email to the NEC (or board members). The NEC then writes the paper. 
After the paper is written, the secretary compile, circulates and distributes the paper to the board members.


Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]

Plan: Find out who are the NEC or board members. #E2 = PEOPLE_DB[who are the NEC or board members]

Plan: the secretary sends an email to the NEC (or board members).The NEC writes the paper. #E3 = sendEmail_chase_gatherPaper[#E2]

Plan: compilePaper #E4 = compilePaper[#E3] 

Plan: After the paper is written, the secretary circulates and distributes the paper to the board members. #E5 = circulatePaper[#E4]


Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.


\nTask: {task}
\nContext: {context}


\nAnswer:""" 


PRB HERE
\nQuestion: {question}
\nContext: {context}

In [155]:
# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt

In [156]:
# simply Converting prompt into p_prompt: ChatPromptTemplate as langchain receives in that form

# p_prompt = ChatPromptTemplate(input_variables=['context', 'question'], 
#                             metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, 
#                             messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=prompt))])


# same result as code above
p_prompt = ChatPromptTemplate.from_messages([("user", prompt)])

Instead of model(llm+rag), test llm only

In [157]:
# # Use LLM + RAG

# model = (
# {"context": retriever | format_docs, "task": RunnablePassthrough()}
# | p_prompt
# | llm
# | StrOutputParser()
# )

# query = "how do I create a board paper?"
# res = model.invoke(query)
# print(res)
# type(res) #str

In [158]:
## TESTING LLM
# task = "how do I create a board paper?"
# res = llm.invoke(task)
# print(res)
# type(res) #langchain_core.messages.ai.AIMessage



In [159]:
task = "how do I create a board paper?"

In [160]:
model = llm

Planner Node

To connect the planner to our graph,
we will create a "get_plan" node 
- accepts the ReWOO state and 
- returns with a state update for the steps and plan_string fields.

### get_plan - call this from langgraph later 

PRB HERE



In [161]:
# model = (
# {"context": retriever | format_docs, "task": RunnablePassthrough()}
# | p_prompt
# | llm
# | StrOutputParser()
# )

In [162]:
#og code

# Regex to match expressions of the form  "Plan: E#... = ...[...]"
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]" 

prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model


def get_plan(state: ReWOO):

    task = state["task"]

    result = planner.invoke({"task": task, "context": retriever, "recursion_limit":100}) #ask agent about the each task/plan again 
    
    # Find all matches in the sample text    
    matches = re.findall(regex_pattern, result.content) 
    # result.content = plan_string e.g. Plan: Find out what is approved agenda. #E1 = RAG+LLM[what is approved agenda]
    # matches = a list =  [Step description, step name(#E1, #E2, ...), tool, input]

    return {"steps": matches, "plan_string": result.content}


In [164]:
prompt_template

ChatPromptTemplate(input_variables=['context', 'task'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'task'], template="For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n\n(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\n(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. \nInput will be a list of board members and agendas.\n\n(3) COMPILE[inp

In [165]:
planner

ChatPromptTemplate(input_variables=['context', 'task'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'task'], template="For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n\n(1) RAG_LLM[input]: A pretrained RAG+LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense + specific knowledge and data especially for Board and secretary role. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\n(2) EMAIL[input]: Worker that send emails to board members to ask write board paper, check status of the return of those email and chase. \nInput will be a list of board members and agendas.\n\n(3) COMPILE[inp

## [2] EXECUTOR

Define the tool execution node

In [166]:
# load plugins & test

from plugins.basic import BasicPlugins

plugin_instance = BasicPlugins()

result = plugin_instance.email_function()
result = plugin_instance.compile_paper()
result = plugin_instance.circulate_paper()
result = plugin_instance.people_db()


result

'Board Members:\n\nMike CEO\n\nmike@mycompany.com\n\nCEO\n\nOlivia Johnson\n\nolivia@mycompany.com\n\nCFO\n\n'

In [167]:
import textwrap


def _get_current_task(state: ReWOO):
    if state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1
    

# initialise plugins
from plugins.basic import BasicPlugins
plugin_instance = BasicPlugins()


prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


output_parser = StrOutputParser()
rag_chain = prompt | model | output_parser

def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1] # state["steps"] = List   
    _results = state["results"] or {}

    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    
    print("[tool_execution] Print selected tool name: ", tool, "/n")

    if tool == "RAG_LLM":

        docs = retriever.get_relevant_documents(tool_input)
        docs_str = format_docs(docs)


        print("[tool_execution] tool input: ", tool_input, "/n")
        print("[tool_execution] retriever res docs: ", docs, "/n")        


        # result = model.invoke(tool_input) #e.g. tool_input = "what is approved agenda"
        result = rag_chain.invoke({"context": docs_str, "question": task, "recursion_limit":100}) #### NOW TEST PASSING DOCS INSTEAD OF RAG_CHAIN

        print("---print pretty---")
        print("\n".join(textwrap.wrap(result, width = 140)))
        print("---wrap test---")

        
        print("[tool_execution] rag invoke result: ", result, "/n")
    elif tool =="PEOPLE_DB":
        result = plugin_instance.people_db()
    elif tool =="EMAIL":
        result = plugin_instance.email_function()
    elif tool =="COMPILE":
        result = plugin_instance.compile_paper()
    elif tool =="CIRCULATE":
        result = plugin_instance.circulate_paper()
    else:
        raise ValueError
    
    _results[step_name] = str(result) # #E1 = xyz
    return {"results": _results}

## [3] Solver 

The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker.

Solver has its own prompt.

In [168]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""

def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = state["results"] or {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)                                      # should I use LLM or RAG + LLM ???? Try just LLM first, will be faster
    return {"result": result.content}

## [4] Define Graph

Our graph defines the workflow. 

Each of the planner, worker(tool executor), and solver modules are added as nodes.

In [169]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [170]:
from langgraph.graph import StateGraph, END

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.set_entry_point("plan")

app = graph.compile()

In [171]:
task

'how do I create a board paper?'

In [172]:
import textwrap



for s in app.stream({"task": task}):
    print(s) #dict
    print("-------")


{'plan': {'steps': [('Understand the task. ', '#E1', 'RAG_LLM', 'What does it mean to create a board paper?'), ('Identify the steps required in creating a board paper. ', '#E2', 'RAG_LLM', 'What are the steps in creating a board paper?'), ('Find out who are the board members. ', '#E3', 'PEOPLE_DB', 'who are the board members'), ('Ask the board members to write their respective parts of the board paper. ', '#E4', 'EMAIL', '#E3'), ('Compile all the separate parts into one board paper. ', '#E5', 'COMPILE', '#E4'), ('Circulate the compiled board paper to the board members for feedback. ', '#E6', 'CIRCULATE', '#E5'), ('After getting feedback, make necessary revisions to the board paper. ', '#E7', 'RAG_LLM', 'How to revise a board paper?'), ('Finalise the board paper and distribute it to the board members. ', '#E8', 'RAG_LLM', 'How to finalize and distribute a board paper?')], 'plan_string': 'Plan: Understand the task. #E1 = RAG_LLM[What does it mean to create a board paper?]\n\nPlan: Identi